In [4]:
import pandas as pd

# Load each file
products_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\Global+Electronics+Retailer\Products.csv")
customers_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\Global+Electronics+Retailer\Customers.csv", encoding='cp775')
sales_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\Global+Electronics+Retailer\Sales.csv")
stores_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\Global+Electronics+Retailer\Stores.csv")
exchange_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\Global+Electronics+Retailer\Exchange_Rates.csv")

# Check headers
print(products_df.columns.tolist())
print(customers_df.columns.tolist())
print(sales_df.columns.tolist())
print(stores_df.columns.tolist())
print(exchange_df.columns.tolist())


['ProductKey', 'Product Name', 'Brand', 'Color', 'Unit Cost USD', 'Unit Price USD', 'SubcategoryKey', 'Subcategory', 'CategoryKey', 'Category']
['CustomerKey', 'Gender', 'Name', 'City', 'State Code', 'State', 'Zip Code', 'Country', 'Continent', 'Birthday']
['Order Number', 'Line Item', 'Order Date', 'Delivery Date', 'CustomerKey', 'StoreKey', 'ProductKey', 'Quantity', 'Currency Code']
['StoreKey', 'Country', 'State', 'Square Meters', 'Open Date']
['Date', 'Currency', 'Exchange']


#### The customers file had some errors while being read. After checking the encoding, it needed to be opened using a different one. 
#### This was because some names contained characters not found in English.

In [7]:
from charset_normalizer import detect

path = r"C:\Users\mgag2\OneDrive\Documentos\Data\Global+Electronics+Retailer\Customers.csv"

with open(path, 'rb') as file:
    result = detect(file.read())
    print(result)  # Check the detected encoding
    encoding = result['encoding']



{'encoding': 'cp775', 'language': 'English', 'confidence': 0.9944}


In [9]:
#Check data with custom function based on describe
from custom_summary import custom_describe
custom_describe(sales_df)

,Column,Type,Count,Unique,Missing,Min,Max,Average
0,Order Number,int64,62884,26326,0,366000.0,2243032.0,1.430905e+06
1,Line Item,int64,62884,7,0,1.0,7.0,2.164207e+00
2,Order Date,object,62884,1641,0,NaN,NaN,NaN
3,Delivery Date,object,13165,1492,49719,NaN,NaN,NaN
4,CustomerKey,int64,62884,11887,0,301.0,2099937.0,1.180797e+06
5,StoreKey,int64,62884,58,0,0.0,66.0,3.180214e+01
6,ProductKey,int64,62884,2492,0,1.0,2517.0,1.125859e+03
7,Quantity,int64,62884,10,0,1.0,10.0,3.144790e+00
8,Currency Code,object,62884,5,0,NaN,NaN,NaN


In [10]:
# Merge sales with products
sales_with_products = pd.merge(sales_df, products_df, on="ProductKey", how="left")

# Ensure all rows are matched
num_unmatched = sales_with_products['ProductKey'].isna().sum()
if num_unmatched == 0:
    print("All products matched!")
else:
    print(f"There are {num_unmatched} unmatched products.")

All products matched!


In [11]:
# Merge result with customers
sales_with_customers = pd.merge(sales_with_products, customers_df, on="CustomerKey", how="left")

# Ensure all rows are matched
num_unmatched = sales_with_products['CustomerKey'].isna().sum()
if num_unmatched == 0:
    print("All customers matched!")
else:
    print(f"There are {num_unmatched} unmatched customers.")

All customers matched!


In [12]:
# Merge result with stores
sales_with_stores = pd.merge(sales_with_customers, stores_df, on="StoreKey", how="left")

# Ensure all rows are matched
num_unmatched = sales_with_stores['StoreKey'].isna().sum()
if num_unmatched == 0:
    print("All stores matched!")
else:
    print(f"There are {num_unmatched} unmatched stores.")

All stores matched!


In [13]:
# Merge result with exchange
final_sales_df = pd.merge(sales_with_stores, exchange_df, left_on=["Order Date","Currency Code"], right_on=["Date","Currency"], how="left")

# Ensure all rows are matched
num_unmatched = final_sales_df['Exchange'].isna().sum()
if num_unmatched == 0:
    print("All exchange prices matched!")
else:
    print(f"There are {num_unmatched} unmatched exchange prices.")

All exchange prices matched!


In [14]:
print(final_sales_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order Number    62884 non-null  int64  
 1   Line Item       62884 non-null  int64  
 2   Order Date      62884 non-null  object 
 3   Delivery Date   13165 non-null  object 
 4   CustomerKey     62884 non-null  int64  
 5   StoreKey        62884 non-null  int64  
 6   ProductKey      62884 non-null  int64  
 7   Quantity        62884 non-null  int64  
 8   Currency Code   62884 non-null  object 
 9   Product Name    62884 non-null  object 
 10  Brand           62884 non-null  object 
 11  Color           62884 non-null  object 
 12  Unit Cost USD   62884 non-null  object 
 13  Unit Price USD  62884 non-null  object 
 14  SubcategoryKey  62884 non-null  int64  
 15  Subcategory     62884 non-null  object 
 16  CategoryKey     62884 non-null  int64  
 17  Category        62884 non-null 

#### Some headers were renamed automatically. Let's use better names for them.

In [16]:
# Rename specific columns
final_sales_df = final_sales_df.rename(columns={"State_x": "Customer State","Country_x": "Customer Country",
    "State_y": "Store State", "Country_y": "Store Country", "Exchange": "Exchange Rate"
})

# Eliminate redundant columns
final_sales_df = final_sales_df.drop(columns=['Date', 'Currency'])

print(final_sales_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order Number      62884 non-null  int64  
 1   Line Item         62884 non-null  int64  
 2   Order Date        62884 non-null  object 
 3   Delivery Date     13165 non-null  object 
 4   CustomerKey       62884 non-null  int64  
 5   StoreKey          62884 non-null  int64  
 6   ProductKey        62884 non-null  int64  
 7   Quantity          62884 non-null  int64  
 8   Currency Code     62884 non-null  object 
 9   Product Name      62884 non-null  object 
 10  Brand             62884 non-null  object 
 11  Color             62884 non-null  object 
 12  Unit Cost USD     62884 non-null  object 
 13  Unit Price USD    62884 non-null  object 
 14  SubcategoryKey    62884 non-null  int64  
 15  Subcategory       62884 non-null  object 
 16  CategoryKey       62884 non-null  int64 

#### Unit Cost USD and Unit Price USD are not float type

In [31]:
# Remove the '$' symbol and convert the columns to numeric values
final_sales_df['Unit Price USD'] = final_sales_df['Unit Price USD'].replace({'$': '', ',': ''}, regex=True).astype(float)
final_sales_df['Unit Cost USD'] = final_sales_df['Unit Cost USD'].replace({'$': '', ',': ''}, regex=True).astype(float)

In [37]:
print(final_sales_df[['Unit Price USD', 'Unit Cost USD']].head())

   Unit Price USD  Unit Cost USD
0           68.00          31.27
1          427.00         141.47
2          665.94         220.64
3          322.00         148.08
4          326.00         166.20


#### Date columns also have an incorrect data type 

In [53]:
# Ensure is datetime
final_sales_df['Order Date'] = pd.to_datetime(final_sales_df['Order Date'], errors='coerce')
final_sales_df['Delivery Date'] = pd.to_datetime(final_sales_df['Delivery Date'], errors='coerce')
final_sales_df['Open Date'] = pd.to_datetime(final_sales_df['Open Date'], errors='coerce')
final_sales_df['Birthday'] = pd.to_datetime(final_sales_df['Birthday'], errors='coerce')

In [55]:
print(final_sales_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Number      62884 non-null  int64         
 1   Line Item         62884 non-null  int64         
 2   Order Date        62884 non-null  datetime64[ns]
 3   Delivery Date     13165 non-null  datetime64[ns]
 4   CustomerKey       62884 non-null  int64         
 5   StoreKey          62884 non-null  int64         
 6   ProductKey        62884 non-null  int64         
 7   Quantity          62884 non-null  int64         
 8   Currency Code     62884 non-null  object        
 9   Product Name      62884 non-null  object        
 10  Brand             62884 non-null  object        
 11  Color             62884 non-null  object        
 12  Unit Cost USD     62884 non-null  float64       
 13  Unit Price USD    62884 non-null  float64       
 14  SubcategoryKey    6288

In [57]:
%store final_sales_df

Stored 'final_sales_df' (DataFrame)
